In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from qutip import qeye, destroy, tensor
from qutip.core.gates import *
from qutip import Qobj
from qutip import *  

# test to see if sigma+ * sigma- is same as proj(1,1)

sigma_minus, sigma_plus = destroy(2), destroy(2).dag()

spm = sigma_plus * sigma_minus


Iq = np.eye(2)

g = basis(2,0)
e = basis(2,1)


def build_projector(vectors, i, j):
    """Build projector |i><j| from the i-th and j-th eigenvectors from the supplied set of vectors."""
    ket = vectors[:, i][:, np.newaxis]
    bra = vectors[:, j][:, np.newaxis].conj().T

    projector = ket @ bra
    return projector

P_00 = build_projector(Iq, 0, 0)
P_11 = build_projector(Iq, 1, 1)



spm


In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from qutip import qeye, destroy, tensor
from qutip.core.gates import *
from qutip import Qobj
from qutip import *  

pi = np.pi

# -----------------------------------------------------------
# Utility functions
# -----------------------------------------------------------

def wc_t(t, args=None):
    """ Initial time, no cavity coupling yet """
    return 1 if t <= T0_1 else 0


def w1_t(t, args=None):
    """ Control envelope for qubit 1. """
    return 1 if T0_1 < t <= T0_1 + T_gate_1 else 0


def w2_t(t, args=None):
    """ Control envelope for qubit 2. """
    return 1 if T0_2 < t <= T0_2 + T_gate_2 else 0


def read_json_to_dict(filename: str) -> dict:
    """ Read a JSON file into a Python dictionary. """
    with open(filename, "r") as f:
        return json.load(f)

def ladder_operators(dim):
    """ Create ladder operators for a given dimension, return them as numpy arrays. """
    a = destroy(dim)
    adag = a.dag()
    return a.full(), adag.full()

def _tensor(*ops):
    """Kronecker product of multiple operators (NumPy arrays)."""
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)
    return result

def build_projector(vectors, i, j):
    """Build projector |i><j| from the i-th and j-th eigenvectors from the supplied set of vectors."""
    ket = vectors[:, i][:, np.newaxis]
    bra = vectors[:, j][:, np.newaxis].conj().T

    projector = ket @ bra
    return projector

def build_qubit_cavity_vibronic_coupling(H_qubit_cavity_coupling, omega_list, gv_list, boson_dim, qubit_number=1):
    
    ### The problem is H_qubit_cavity_coupling is in the full Hilbert space
    ### but we want it in the qubit-cavity subspace only

    # try tracing out extra degree of freedom
    if qubit_number == 1:
        H_qubit_cavity_coupling = H_qubit_cavity_coupling.ptrace([0,1])
    elif qubit_number == 2:
        H_qubit_cavity_coupling = H_qubit_cavity_coupling.ptrace([1,2])

    # this has polariton dim bc we traced out other degrees of freedom
    H_coup_np = H_qubit_cavity_coupling.full()
    polariton_dim = H_coup_np.shape[0]
    vals, vecs = np.linalg.eigh(H_coup_np)

    # operators on the local bosonic space
    a_np, adag_np = ladder_operators(boson_dim)
    n_np = adag_np @ a_np
    Iq_np = np.eye(2, dtype=complex)
    Iv_np = np.eye(boson_dim, dtype=complex)

    # total Hilbert space if qubit_number == 1: polariton ⊗ qubit2 ⊗ vib1 ⊗ vib2
    # and if                 qubit_number == 2: qubit1 ⊗ polariton ⊗ vib1 ⊗ vib2
    total_dim = polariton_dim * 2 * boson_dim * boson_dim
    H_vib = np.zeros((total_dim, total_dim), dtype=complex)
    H_vib_coup = np.zeros_like(H_vib)

    for i in range(polariton_dim):
        Proj_ii = build_projector(vecs, i, i)
        if qubit_number==1:
            op_n = _tensor(Proj_ii, Iq_np, n_np, Iv_np)
        elif qubit_number==2:
            op_n = _tensor(Iq_np, Proj_ii, Iv_np, n_np)
        
        H_vib += omega_list[i] * op_n

        if i > 0:
            Proj_0i = build_projector(vecs, 0, i) + build_projector(vecs, i, 0)

            if qubit_number==1:
                op_c = _tensor(Proj_0i, Iq_np, (a_np + adag_np), Iv_np)
            elif qubit_number==2:
                op_c = _tensor(Iq_np, Proj_0i, Iv_np, (a_np + adag_np))

            H_vib_coup = gv_list[i] * op_c

    qubit_dims = [2, boson_dim, 2, boson_dim, boson_dim]
    return Qobj(H_vib + H_vib_coup, dims=[qubit_dims, qubit_dims])
    
# -----------------------------------------------------------
# Load system parameters
# -----------------------------------------------------------
LiH_params = read_json_to_dict("LiH_params.json")

Nf = 3   # bosonic cutoff
# local operators
sz = sigmaz()
sm = destroy(2)
sp = sm.dag()
nq = sp * sm

am = destroy(Nf)
ap = am.dag()
nc = ap * am

# identities
Iq = qeye(2)
Ic = qeye(Nf)
Iv = qeye(Nf)

# fixed frequencies
omega_q = LiH_params["w_q1"]
omega_c = omega_q  # enforce resonance
omega_v = LiH_params["w_vib1"]
mu_eg = LiH_params["qubit_1_dipole_moments"]["mu_eg"]
mu_ee = LiH_params["qubit_1_dipole_moments"]["mu_e"]
mu_gg = LiH_params["qubit_1_dipole_moments"]["mu_g"]

# define list of omega values for vibrational frequencies of polariton states, there should be 2 * Nf of these values
# for Nf = 3, these are ordered as frequency of coupled gs, frequency of LP, frequency of UP, frequency of S0, frequency of S1, frequency of S1 
omega_vib_list = [0.006, # omega on gs
                  0.006, # omega on LP
                  0.006, # omega on UP
                  0.006, # omega on S0
                  0.006, # omega on S1
                  0.006] # omega on S1 

# gv values for transitions between grouna and and excited polariton states
# for Nf = 3, these are ordered as gv for coupled gs to LP, coupled gs to UP, coupled gs to S0 (0), coupled gs to S1, coupled gs to S1
gv_list = [0.00, # not used
           0.00, # gs to LP
           0.00, # gs to UP
           0.00, # gs to S0
           0.0, # gs to S1
           0.01] # gs to S1           


# state ordering is q1, cav, q2, v1, v2

# initial state: |e,0,g,0,0>
psi0 = tensor(basis(2,1), basis(Nf,0),
              basis(2,0), basis(Nf,0), basis(Nf,0))

# ideal Bell target
rho_qubits_ideal = ket2dm(
    tensor(phasegate(0), phasegate(pi/2)) *
    sqrtiswap() *
    tensor(basis(2,1), basis(2,0))
)

# time discretization
tlist = np.linspace(0, 1400, 5000)

# bare Hamiltonian (independent of gc, gv)
H_q1 = tensor(-omega_q/2 * sz, Ic, Iq, Iv, Iv)
H_cav = tensor(Iq, omega_c * nc, Iq, Iv, Iv)
H_q2 = tensor(Iq, Ic,  -omega_q/2 * sz, Iv, Iv)
H_v1 = tensor(Iq, Ic, Iq, omega_v * nc, Iv)
H_v2 = tensor(Iq, Ic, Iq, Iv, omega_v * nc)
H_q1_v1 = gv_list[4] * tensor(sp * sm, Ic, Iq, (am + ap), Iv)
H_q2_v2 = gv_list[4] * tensor(Iq, Ic, sp * sm, Iv, (am + ap))



# -----------------------------------------------------------
# Parameter scan
# -----------------------------------------------------------
lv = 0.01
gc_fixed = np.sqrt( omega_c / 2) * lv * np.abs(mu_eg)  # <-- gc_fixed determined by lv, omega_c, and mu_eg

# dipole operator (fixed by lv) - used in Rabi and PF couplings
d_matrix_fixed = lv * np.array([[mu_gg, mu_eg], [mu_eg, mu_ee]])
d_qobj = Qobj(d_matrix_fixed)

# JC bilinear coupling
H_q1_cav_JC = gc_fixed * tensor(sp, am, Iq, Iv, Iv) + gc_fixed * tensor(sm, ap, Iq, Iv, Iv) 
H_q2_cav_JC = gc_fixed * tensor(Iq, am, sp, Iv, Iv) + gc_fixed * tensor(Iq, ap, sm, Iv, Iv)

H_q1_vib1_coupled = build_qubit_cavity_vibronic_coupling(H_q1 + H_cav + H_q1_cav_JC, omega_vib_list, gv_list, Nf, qubit_number=1)
H_q2_vib2_coupled = build_qubit_cavity_vibronic_coupling(H_q2 + H_cav + H_q2_cav_JC, omega_vib_list, gv_list, Nf, qubit_number=2)

print(H_q1_vib1_coupled)

# Rabi bilinear coupling
#H_q1_cav_Rabi = np.sqrt(omega_c / 2) * tensor(d_qobj, (am + ap), Iq, Iv, Iv)
#H_q2_cav_Rabi = np.sqrt(omega_c / 2) * tensor(Iq, (am + ap), d_qobj, Iv, Iv, )

# Pauli-Fierz is Rabi + DSE
#H_q1_cav_PF = H_q1_cav_Rabi + 1/2 * tensor(d_qobj * d_qobj, Ic, Iq, Iv, Iv)
#H_q2_cav_PF = H_q2_cav_Rabi + 1/2 * tensor(Iq, Ic, d_qobj * d_qobj, Iv, Iv)


# gate times fixed by gc
global T0_1, T0_2, T_gate_1, T_gate_2
T0_1 = 20
T_gate_1 = pi / (4 * np.abs(gc_fixed))
T0_2 = T0_1 + T_gate_1
T_gate_2 =  pi / (2 * np.abs(gc_fixed))

H_uncoupled = H_q1 + H_cav + H_q2 + H_v1 + H_v2 + H_q1_v1 + H_q2_v2
H_q1_cav_coupled = H_q1 + H_cav + H_q1_cav_JC + H_q2 + H_v1 + H_v2 + H_q1_vib1_coupled + H_q2_v2
H_q2_cav_coupled = H_q1 + H_cav + H_q2_cav_JC + H_q2 + H_v1 + H_v2 + H_q1_v1 + H_q2_vib2_coupled
    
H_t_PF = [[H_uncoupled, wc_t], [H_q1_cav_coupled, w1_t], [H_q2_cav_coupled, w2_t]]


res_PF = mesolve(H_t_PF, psi0, tlist, [], e_ops=[])
rho_final_PF = res_PF.states[-1]



rho_qubits_PF = ptrace(rho_final_PF, [0,2])


print("Fidelity of PF qubit state with ideal Bell state:", fidelity(rho_qubits_PF, rho_qubits_ideal))
print("concurrence of PF qubit state:", concurrence(rho_qubits_PF))

print(rho_qubits_PF)

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from qutip import *
from scipy.optimize import differential_evolution, minimize

# -----------------------------------------------------------
# Objective Function
# -----------------------------------------------------------
def objective(params, LiH_params, psi0, rho_qubits_ideal, tlist, Nf, omega_vib_list, history=None):
    """
    Objective: minimize 1 - fidelity with ideal Bell state.
    params = [gv_1, gv_2, gv_5, T_gate_1, T_gate_2]
    """
    gv_list = np.zeros_like(omega_vib_list)
    gv_list[0] = 0.0
    gv_list[1] = params[0] # gs to LP
    gv_list[2] = params[1] # gs to UP
    gv_list[3] = 0.0       # gs to S0
    gv_list[4] = params[2] # gs to S1
    gv_list[5] = params[2] # gs to S1 
    
    T_gate_1, T_gate_2 = params[3], params[4]

    global T0_1, T0_2
    T0_1 = 20
    T0_2 = T0_1 + T_gate_1

    # System constants
    omega_q = LiH_params["w_q1"]
    omega_c = omega_q
    omega_v = LiH_params["w_vib1"]
    mu_eg = LiH_params["qubit_1_dipole_moments"]["mu_eg"]
    mu_ee = LiH_params["qubit_1_dipole_moments"]["mu_e"]
    mu_gg = LiH_params["qubit_1_dipole_moments"]["mu_g"]
    lv = 0.01

    gc_fixed = np.sqrt(omega_c / 2) * lv * np.abs(mu_eg)
    d_matrix_fixed = lv * np.array([[mu_gg, mu_eg], [mu_eg, mu_ee]])
    d_qobj = Qobj(d_matrix_fixed)

    # Local operators
    sz, sm, sp = sigmaz(), destroy(2), destroy(2).dag()
    nq = sp * sm
    am, ap = destroy(Nf), destroy(Nf).dag()
    nc = ap * am
    Iq, Ic, Iv = qeye(2), qeye(Nf), qeye(Nf)

    # Build Hamiltonians
    H_q1 = tensor(-omega_q / 2 * sz, Ic, Iq, Iv, Iv)
    H_cav = tensor(Iq, omega_c * nc, Iq, Iv, Iv)
    H_q2 = tensor(Iq, Ic, -omega_q / 2 * sz, Iv, Iv)
    H_v1 = tensor(Iq, Ic, Iq, omega_v * nc, Iv)
    H_v2 = tensor(Iq, Ic, Iq, Iv, omega_v * nc)

    H_q1_cav_JC = gc_fixed * (tensor(sp, am, Iq, Iv, Iv) + tensor(sm, ap, Iq, Iv, Iv))
    H_q2_cav_JC = gc_fixed * (tensor(Iq, am, sp, Iv, Iv) + tensor(Iq, ap, sm, Iv, Iv))

    H_q1_v1 = gv_list[4] * tensor(sp * sm, Ic, Iq, (am + ap), Iv)
    H_q2_v2 = gv_list[4] * tensor(Iq, Ic, sp * sm, Iv, (am + ap))

    # Vibronic coupling Hamiltonians
    H_q1_vib1_coupled = build_qubit_cavity_vibronic_coupling(
        H_q1 + H_cav + H_q1_cav_JC, omega_vib_list, gv_list, Nf, qubit_number=1
    )
    H_q2_vib2_coupled = build_qubit_cavity_vibronic_coupling(
        H_q2 + H_cav + H_q2_cav_JC, omega_vib_list, gv_list, Nf, qubit_number=2
    )

    # Time-dependent Hamiltonian parts
    def wc_t(t, args=None): return 1 if t <= T0_1 else 0
    def w1_t(t, args=None): return 1 if T0_1 < t <= T0_1 + T_gate_1 else 0
    def w2_t(t, args=None): return 1 if T0_2 < t <= T0_2 + T_gate_2 else 0

    H_uncoupled = H_q1 + H_cav + H_q2 + H_v1 + H_v2 + H_q1_v1 + H_q2_v2
    H_q1_cav_coupled = H_q1 + H_cav + H_q1_cav_JC + H_q2 + H_v1 + H_v2 + H_q1_vib1_coupled + H_q2_v2
    H_q2_cav_coupled = H_q1 + H_cav + H_q2_cav_JC + H_q2 + H_v1 + H_v2 + H_q1_v1 + H_q2_vib2_coupled

    H_t_PF = [[H_uncoupled, wc_t], [H_q1_cav_coupled, w1_t], [H_q2_cav_coupled, w2_t]]

    try:
        res_PF = mesolve(H_t_PF, psi0, tlist, [], e_ops=[])
        rho_final_PF = res_PF.states[-1]
        rho_qubits_PF = ptrace(rho_final_PF, [0, 2])
        fid = fidelity(rho_qubits_PF, rho_qubits_ideal)
        loss = 1 - fid
    except Exception as e:
        print(f"Simulation failed: {e}")
        loss = 10.0  # penalty for failed evolution

    if history is not None:
        history.append(1 - loss)  # store fidelity

    return loss


# -----------------------------------------------------------
# Logging + Plotting Helpers
# -----------------------------------------------------------
class OptimizationLogger:
    def __init__(self):
        self.history = []

    def callback(self, xk, convergence=None):
        # called after each differential evolution iteration
        print(f"Iteration {len(self.history)} | Best Fidelity = {self.history[-1]:.6f}")
        return False  # continue optimization

    def plot(self, title="Optimization Progress"):
        plt.figure(figsize=(7, 4))
        plt.plot(self.history, "-o", label="Best Fidelity")
        plt.xlabel("Iteration")
        plt.ylabel("Fidelity")
        plt.title(title)
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()

    def save(self, filename="optimization_results.json"):
        with open(filename, "w") as f:
            json.dump({"fidelity_history": self.history}, f, indent=2)
        print(f"Saved optimization history to {filename}")


# -----------------------------------------------------------
# Run Global + Local Optimization
# -----------------------------------------------------------
def run_optimization(LiH_params, psi0, rho_qubits_ideal, tlist, Nf, omega_vib_list):
    logger = OptimizationLogger()

    bounds = [
        (1e-5, 1.0),   # g to LP
        (1e-5, 1.0),  # g to UP
        (1e-5, 1.0),  #  g to S1
        (10, 400),    # T_gate_1
        (20, 800),    # T_gate_2
    ]

    # --- Differential Evolution (Global Search)
    result_de = differential_evolution(
        objective,
        bounds,
        args=(LiH_params, psi0, rho_qubits_ideal, tlist, Nf, omega_vib_list, logger.history),
        maxiter=25,
        tol=1e-2,
        polish=False,
        updating="deferred",
        workers=1,
        callback=logger.callback
    )

    print("\nGlobal search complete.")
    print("Best parameters (DE):", result_de.x)
    print("Best fidelity:", 1 - result_de.fun)

    logger.plot(title="Differential Evolution Progress")

    # --- L-BFGS-B (Local Refinement)
    result_lbfgs = minimize(
        objective,
        result_de.x,
        args=(LiH_params, psi0, rho_qubits_ideal, tlist, Nf, omega_vib_list, logger.history),
        method="L-BFGS-B",
        bounds=bounds,
        options={"maxiter": 50, "disp": True}
    )

    final_fid = 1 - result_lbfgs.fun
    print("\nLocal refinement complete.")
    print("Refined parameters (L-BFGS-B):", result_lbfgs.x)
    print("Final fidelity:", final_fid)

    logger.plot(title="Combined Optimization Progress")
    logger.save()

    return result_lbfgs, logger


# -----------------------------------------------------------
# Example Usage
# -----------------------------------------------------------
result, logger = run_optimization(LiH_params, psi0, rho_qubits_ideal, tlist, Nf, omega_vib_list)


In [ ]:
import matplotlib.ticker as ticker
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(gc_fixed / gv_values, fidelities, 'o-', label="Fidelity")
ax.plot(gc_fixed / gv_values, concurrences, 's-', label="Concurrence")

ax.set_xlabel("gv")
ax.set_ylabel("Value")
ax.set_title(f"Fidelity & Concurrence vs gv (gc fixed = {gc_fixed})")
ax.legend()
ax.grid(True)

# Force scientific notation on x-axis
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
#ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.3e'))
ax.loglog()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# === Global Styling (applies to all figures) ===
plt.rcParams.update({
    "figure.figsize": (6, 4),          # consistent size
    "font.family": "serif",            # journal-style (use "Times New Roman" if required)
    "font.size": 12,                   # base font size
    "axes.labelsize": 12,              # axis labels
    "axes.titlesize": 13,              # title
    "axes.titleweight": "bold",
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 2,              # thicker lines
    "lines.markersize": 6,
    "grid.linestyle": "--",
    "grid.linewidth": 0.6,
    "grid.alpha": 0.7
})

# === Your plot ===
fig, ax = plt.subplots()

ax.plot(gc_fixed / gv_values, fidelities, 'o-', color="C0", label="Fidelity")
ax.plot(gc_fixed / gv_values, concurrences, 's--', color="C1", label="Concurrence")

ax.set_xlabel(r"$g_c / g_v$")
ax.set_ylabel("Value")
ax.set_title(f"Fidelity & Concurrence vs $g_c / g_v$\n($g_c$ fixed = {gc_fixed})")

ax.legend(frameon=True, loc="best")
ax.grid(True, which="both")

# Force scientific notation + log scales
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax.set_xscale("log")
ax.set_yscale("log")
#ax.set_xlim(1,5)
plt.tight_layout()

# === Save as vector graphics for publication ===
plt.savefig("fidelity_concurrence_vs_gv.pdf", bbox_inches="tight")   # vector, scalable
plt.savefig("fidelity_concurrence_vs_gv.png", dpi=600, bbox_inches="tight")  # high-res raster

plt.show()

